In [2]:
!pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
!pip install datasets
from datasets import load_metric
!pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 115.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
train = pd.read_csv(r'/content/drive/MyDrive/train_5.csv', encoding='utf_8_sig')
test = pd.read_csv(r'/content/drive/MyDrive/test_5.csv', encoding='utf_8_sig')


In [24]:
train

,label,text,diff,embeddings
0,negative,Fixed link errors,diff --git a/src/irisnet/CMakeLists.txt b/src/...,"[-0.06291048, 0.0064971885, -0.033377588, 0.07..."
1,positive,Check types to avoid invalid reads/writes.,diff --git a/src/file.c b/src/file.c\nindex 4d...,"[-0.05857518, -0.048272554, -0.049325384, 0.19..."
2,positive,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/png.c b/coders/png.c\ninde...,"[-0.11584581, 0.15159129, 0.048209693, 0.01636..."
3,positive,XSS 취약점 수정,diff --git a/adm/boardgroup_form.php b/adm/boa...,"[-0.07697539, 0.023694366, -0.019904647, -0.03..."
4,negative,Change distribution URL.\n\ngit-svn-id: https:...,diff --git a/c/xml-security-c.spec b/c/xml-sec...,"[-0.09832679, 0.33839914, 0.044457793, -0.0695..."
5,negative,Merge pull request #2426 from alvarobartt/deve...,diff --git a/nltk/tag/stanford.py b/nltk/tag/s...,"[-0.076871864, 0.17812185, 0.0046105892, 0.096..."
6,negative,2021 license,diff --git a/Gruntfile.js b/Gruntfile.js\ninde...,"[-0.102409706, 0.16134381, 0.036148887, -0.122..."
7,positive,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...,"[-0.20243266, 0.24396461, 0.03315163, -0.00955..."
8,negative,Upgrade: Bump grape from 1.3.3 to 1.5.0\n\nBum...,diff --git a/Gemfile.lock b/Gemfile.lock\ninde...,"[-0.11363002, 0.23507677, 0.038955946, 0.08096..."
9,negative,Merge branch '8.5' into 9.5,diff --git a/src/Framework/Assert.php b/src/Fr...,"[-0.16887848, 0.015072884, -0.013766231, -0.01..."


In [19]:
len(train)

10

In [20]:
train['label'].value_counts()

negative    6
positive    4
Name: label, dtype: int64

In [21]:
test['label'].value_counts()

negative    6341
positive    3761
Name: label, dtype: int64

In [22]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [28]:
train['embeddings'] = train['text'].apply(lambda x: model.encode(str(x)))
test['embeddings'] = test['text'].apply(lambda x: model.encode(str(x)))

In [29]:
test

,label,text,diff,embeddings
0,negative,Copyright header.,diff --git a/src/main/java/org/projectforge/ad...,"[0.100105606, -0.13413377, -0.0052697286, 0.03..."
1,negative,:art: 部分类增加实现序列化接口,diff --git a/weixin-java-mp/src/main/java/me/c...,"[0.116996, -0.06432121, -0.036785085, 0.024098..."
2,negative,Update copyright to 2020,diff --git a/README.rst b/README.rst\nindex 76...,"[0.055652667, 0.3745133, -0.0034549295, -0.130..."
3,negative,test case for testing old knownnodes pickle\n-...,diff --git a/src/network/knownnodes.py b/src/n...,"[-0.0800946, 0.14758542, 0.027495513, 0.066439..."
4,negative,Introspection fixes.,diff --git a/gsf/gsf-docprop-vector.c b/gsf/gs...,"[-0.014490347, 0.27142954, -0.027338713, 0.000..."
...,...,...,...,...
10097,negative,Fix alignment,diff --git a/wwwroot/cgi-bin/awstats.pl b/wwwr...,"[-0.12206936, -0.13066287, -0.10280945, 0.0171..."
10098,negative,hwdb: Chuwi Hi12 (#17042)\n\nDifferent bios ve...,diff --git a/hwdb.d/60-sensor.hwdb b/hwdb.d/60...,"[-0.06266644, -0.12892303, 0.04864201, -0.1532..."
10099,negative,Clarify text regarding enabling/disabling modu...,diff --git a/config-templates/config.php b/con...,"[0.054796834, -0.23509134, -0.0317173, 0.11958..."
10100,negative,Merge branch 'documentation-update' into 'mast...,diff --git a/docs/api/index.rst b/docs/api/ind...,"[-0.12838669, 0.079408325, -0.011249395, 0.032..."


In [30]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [31]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [32]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))

Logistic Regression Accuracy: 0.6899623836863987
Logistic Regression Precision: 0.6815428044124127
Logistic Regression Recall: 0.6899623836863987
Logistic Regression Recall: 0.6641527018666288
